##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# XNNPACK を使用したオンデバイスの推論のプルーニング

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_for_on_device_inference">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/pruning/pruning_for_on_device_inference.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/model-optimization/tensorflow_model_optimization/g3doc/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

[XNNPACK](https://github.com/google/XNNPACK) を使用したオンデバイスの推論のレイテンシーを改善するための Keras 重みプルーニングのガイドへようこそ。

このガイドでは、新しく導入された `tfmot.sparsity.keras.PruningPolicy` API と[XNNPACK スパース推論](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference)を使用して最新の CPU で畳み込みモデルを高速化する方法を実演します。

このガイドでは、モデル作成プロセスの次の手順について説明します。

- 密なベースラインを構築してトレーニングする
- プルーニングを使ってモデルを微調整する
- TFLite に変換する
- オンデバイスのベンチマーク

このガイドでは、プルーニングを使用したモデルの微調整のベストプラクティスについては説明しません。このトピックの詳細については、[包括的なガイド](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md)をご覧ください。

## セットアップ

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile

import tensorflow as tf
import numpy as np

from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

%load_ext tensorboard

## 密なモデルを構築してトレーニングする

[CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) データセットの分類タスク用に、単純なベースライン CNN を構築してトレーニングします。

In [ ]:
# Load CIFAR10 dataset.
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train[:90%]', 'train[90%:]', 'test'],
    as_supervised=True,
    with_info=True,
)

# Normalize the input image so that each pixel value is between 0 and 1.
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.image.convert_image_dtype(image, tf.float32), label

# Load the data in batches of 128 images.
batch_size = 128
def prepare_dataset(ds, buffer_size=None):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.cache()
  if buffer_size:
    ds = ds.shuffle(buffer_size)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  return ds

ds_train = prepare_dataset(ds_train,
                           buffer_size=ds_info.splits['train'].num_examples)
ds_val = prepare_dataset(ds_val)
ds_test = prepare_dataset(ds_test)

# Build the dense baseline model.
dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Compile and train the dense model for 10 epochs.
dense_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

dense_model.fit(
  ds_train,
  epochs=10,
  validation_data=ds_val)

# Evaluate the dense model.
_, dense_model_accuracy = dense_model.evaluate(ds_test, verbose=0)

## 疎なモデルを構築する

[包括的なガイド](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md)の手順を使用して、`tfmot.sparsity.keras.prune_low_magnitude` 関数を適用します。パラメータは、プルーニングによるオンデバイスの高速化（`tfmot.sparsity.keras.PruneForLatencyOnXNNPack`）を対象とします。

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after after 5 epochs.
end_epoch = 5

num_iterations_per_epoch = len(ds_train)
end_step =  num_iterations_per_epoch * end_epoch

# Define parameters for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.25,
                                                               final_sparsity=0.75,
                                                               begin_step=0,
                                                               end_step=end_step),
      'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
}

# Try to apply pruning wrapper with pruning policy parameter.
try:
  model_for_pruning = prune_low_magnitude(dense_model, **pruning_params)
except ValueError as e:
  print(e)

`prune_low_magnitude` を呼び出すと、`ValueError` になり、`Could not find a GlobalAveragePooling2D layer with keepdims = True in all output branches` というメッセージが表示されます。このメッセージは、モデルのプルーニングがポリシー `tfmot.sparsity.keras.PruneForLatencyOnXNNPack` でサポートされていないことを示しています。具体的には、レイヤー `GlobalAveragePooling2D` にはパラメータ `keepdims = True` が必要です。修正して、`prune_low_magnitude` 関数を再度適用します。

In [ ]:
fixed_dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(keepdims=True),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Use the pretrained model for pruning instead of training from scratch.
fixed_dense_model.set_weights(dense_model.get_weights())

# Try to reapply pruning wrapper.
model_for_pruning = prune_low_magnitude(fixed_dense_model, **pruning_params)

`prune_low_magnitude` の呼び出すとエラーは発生しません。つまり、モデルは `tfmot.sparsity.keras.PruneForLatencyOnXNNPack` ポリシーで完全にサポートされており、[XNNPACK スパース推論](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference)を使用して高速化できます。

### 疎なモデルの微調整

[プルーニングの例](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras.md)に従って、密なモデルの重みを使用して疎なモデルを微調整します。25％ のスパース性（重みの 25％ がゼロに設定されている）でモデルの微調整を開始し、75％ のスパース性で終了します。

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

model_for_pruning.fit(
  ds_train,
  epochs=15,
  validation_data=ds_val,
  callbacks=callbacks)

# Evaluate the dense model.
_, pruned_model_accuracy = model_for_pruning.evaluate(ds_test, verbose=0)

print('Dense model test accuracy:', dense_model_accuracy)
print('Pruned model test accuracy:', pruned_model_accuracy)

ログには、レイヤーごとのスパース性の進行状況が示されます。

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

プルーニングによる微調整後、テストの精度は、密なモデルと比較して適度に改善（43％ から 44％）しました。[TFLite ベンチマーク](https://www.tensorflow.org/lite/performance/measurement)を使用してオンデバイスのレイテンシを比較してみましょう。

## モデルの変換とベンチマーク

プルーニングされたモデルを TFLite に変換するには、`strip_pruning` 関数を使用して、`PruneLowMagnitude` ラッパーを元のレイヤーに置き換える必要があります。また、プルーニングされたモデル（`model_for_pruning`）の重みはほとんどゼロであるため、最適化 `tf.lite.Optimize.EXPERIMENTAL_SPARSITY` を適用して、結果の TFLite モデルを効率的に保存できます。この最適化フラグは、密なモデルには必要ありません。

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(dense_model)
dense_tflite_model = converter.convert()

_, dense_tflite_file = tempfile.mkstemp('.tflite')
with open(dense_tflite_file, 'wb') as f:
  f.write(dense_tflite_model)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.EXPERIMENTAL_SPARSITY]
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

[TFLite モデルベンチマークツール](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/tools/benchmark)の指示に従って、ツールを作成し、高密度でプルーニングされた TFLite モデルと共に Android デバイスにアップロードし、デバイスで両方のモデルをベンチマークします。

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/dense_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/pruned_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

Pixel 4 のベンチマークでは、平均推論時間は密なモデルでは *17us*、プルーニングされたモデルでは *12us* でした。オンデバイスのベンチマークは、このような小さなモデルでも、レイテンシが明らかに **5us**（**30％**）改善されていることを示しています。私たちの経験では、[ MobileNetV3](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v3) や [EfficientNet-lite](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet/lite) に基づくと、より大きなモデルでも同様のパフォーマンス向上が示されます。スピードアップは、モデル全体に対する 1x1 畳み込みの相対的な寄与に基づいて異なります。


## 結論

このチュートリアルでは、TF MOT API と XNN Pack の新機能を使用して、オンデバイスのパフォーマンスを高速化するために疎なモデルを作成する方法を実演しました 。これらの疎なモデルは、密なモデルよりも小さく、高速ですが、品質は密のモデルと同等または上回る場合もあります。

この新しい機能は、モデルをデバイスにデプロイする際に特に重要となります。ぜひ試してみてください。